In [1]:
import z3
import json
import pprint
import numpy as np
import pandas as pd

with open ('simple.json') as file:
    data = json.load(file)

#------------SETUP-------------------------------

# List of Resource Types
resourceTypes = set([j for i in data for j in data[i]['resources'] ])

# Initialize data structures
nodeList = list()
appList = list()
nlappList = list() # List of apps that are dependencies
prAppList = list() # List of primary apps that have dependencies
a2a = []
n2n = []

# Node and App lists
for i in data:
    if data[i]['type']=='node':
        nodeList.append(i)
    if data[i]['type']=='app':
        appList.append(i)
        if data[i]['dependencies']['nonlocal']:
            prAppList.append(i)
            for dep in data[i]['dependencies']['nonlocal']:
                nlappList.append(dep)
print("Dependency apps: %s" %(nlappList))
print("Primary apps: %s" %(prAppList))
# app to app dependency matrix            
for app in appList:
    nl_deps = data[app]['dependencies']['nonlocal']
    deps = []
    for dep in appList:
        if dep in nl_deps:
            deps.append(1)
        else: 
            deps.append(0)
    a2a.append(deps)
a2a_df = pd.DataFrame(a2a, index=appList, columns=appList)        

# node to node latency matrix
n2n = []
for node in nodeList:
    id = data[node]['id']
    temp = []    
    for n2 in nodeList:
        id2 = data[n2]['id']
        lat = data[node]['latencies']['n%s%s' %(id, id2)]
        temp.append(lat)
    n2n.append(temp)
n2n_df = pd.DataFrame(n2n, index=nodeList, columns=nodeList)  

# node resource matrix
rpn= [[data[j]['resources'][k] for j in nodeList] for k in resourceTypes]
rpn_df = pd.DataFrame(rpn, index=resourceTypes, columns=nodeList)     
    
# app resource matrix
rpa= [[data[i]['resources'][k] for i in appList] for k in resourceTypes]
rpa_df = pd.DataFrame(rpa, index=resourceTypes, columns=appList)     

# app to node placment matrix
a2n = [ [ z3.Int("a%s%s" % (i+1, j+1)) for j in range(len(nodeList)) ]
            for i in range(len(appList))]
a2n_df = pd.DataFrame(a2n, index=appList, columns=nodeList) 

print("n2n: \n%s" %n2n_df)
print("a2a: \n%s" %a2a_df)
print("rpn: \n%s" %rpn_df)
print("rpa: \n%s" %rpa_df)

print("a2n: \n%s" %a2n_df)

Dependency apps: ['APP2']
Primary apps: ['APP1']
n2n: 
       FSSN3  FSSN5  FSSN1  FSSN2  FSSN4
FSSN3      0      8      4      5      7
FSSN5      8      0      6      7      9
FSSN1      4      6      0      3      5
FSSN2      5      7      3      0      6
FSSN4      7      9      5      6      0
a2a: 
      APP1  APP2
APP1     0     1
APP2     0     0
rpn: 
        FSSN3  FSSN5  FSSN1  FSSN2  FSSN4
memory    512    512   1024    512    512
rpa: 
        APP1  APP2
memory   256   128
a2n: 
     FSSN3 FSSN5 FSSN1 FSSN2 FSSN4
APP1   a11   a12   a13   a14   a15
APP2   a21   a22   a23   a24   a25


In [2]:
a2n_df

,FSSN3,FSSN5,FSSN1,FSSN2,FSSN4
APP1,a11,a12,a13,a14,a15
APP2,a21,a22,a23,a24,a25


In [3]:
n2n_df

,FSSN3,FSSN5,FSSN1,FSSN2,FSSN4
FSSN3,0,8,4,5,7
FSSN5,8,0,6,7,9
FSSN1,4,6,0,3,5
FSSN2,5,7,3,0,6
FSSN4,7,9,5,6,0


In [4]:
a2a_df

,APP1,APP2
APP1,0,1
APP2,0,0


In [5]:
rpn_df

,FSSN3,FSSN5,FSSN1,FSSN2,FSSN4
memory,512,512,1024,512,512


In [6]:
rpa_df

,APP1,APP2
memory,256,128


In [7]:
#------------Constraints-------------------------------
# variables in a2n matrix must be binary

a2n_domain = [z3.Or(a2n_df.loc[app, node]==0, a2n_df.loc[app, node]==1) \
 for app in nlappList \
 for node in nodeList]

# An intial deployment of apps that have dependencies
a2n_df.loc['APP1'] = 1
#a2n_df.loc['APP1', 'FSSN2'] = 1
#a2n_df.loc['APP1', 'FSSN3'] = 1
print (a2n_df)
a2n_domain


     FSSN3 FSSN5 FSSN1 FSSN2 FSSN4
APP1     1     1     1     1     1
APP2   a21   a22   a23   a24   a25


[Or(a21 == 0, a21 == 1),
 Or(a22 == 0, a22 == 1),
 Or(a23 == 0, a23 == 1),
 Or(a24 == 0, a24 == 1),
 Or(a25 == 0, a25 == 1)]

In [8]:
# Max capacity constraint
# Resources required by apps deployed on a node cannot exceed that nodes resources
app_util = rpa_df.dot(a2n_df)
print(app_util)
node_util = rpn_df - app_util
print(node_util)
rsrc_constraint = [node_util.loc[rsrc,node]>=0 \
                   for rsrc in resourceTypes \
                   for node in nodeList]
rsrc_constraint

                FSSN3          FSSN5          FSSN1          FSSN2  \
memory  256 + 128*a21  256 + 128*a22  256 + 128*a23  256 + 128*a24   

                FSSN4  
memory  256 + 128*a25  
                        FSSN3                  FSSN5                   FSSN1  \
memory  512 - (256 + 128*a21)  512 - (256 + 128*a22)  1024 - (256 + 128*a23)   

                        FSSN2                  FSSN4  
memory  512 - (256 + 128*a24)  512 - (256 + 128*a25)  


[512 - (256 + 128*a21) >= 0,
 512 - (256 + 128*a22) >= 0,
 1024 - (256 + 128*a23) >= 0,
 512 - (256 + 128*a24) >= 0,
 512 - (256 + 128*a25) >= 0]

In [9]:
# Must Deploy Dependencies constraint
# Assuming the primary app is deployed, all of it's dependencies must be deployed
mdd = [a2n_df.loc[dep].sum() > 0 for dep in nlappList]
mdd

[a21 + a22 + a23 + a24 + a25 > 0]

In [10]:
print([n2n_df.loc[src,dst] for src in nodeList for dst in nodeList])

[0, 8, 4, 5, 7, 8, 0, 6, 7, 9, 4, 6, 0, 3, 5, 5, 7, 3, 0, 6, 7, 9, 5, 6, 0]


In [11]:
n2n_df

,FSSN3,FSSN5,FSSN1,FSSN2,FSSN4
FSSN3,0,8,4,5,7
FSSN5,8,0,6,7,9
FSSN1,4,6,0,3,5
FSSN2,5,7,3,0,6
FSSN4,7,9,5,6,0


In [12]:
# Latency 
numNodes = len(nodeList) # number of FSSNs
markings = list() # The equations for each possible placement of the dependencies

for srcNode in nodeList:
    print (srcNode)
    for prApp in prAppList:
        if a2n_df.loc[app, srcNode] == 1:            
            ### ITERATE THROUGH ALL DEPENDENCIES
            # iterate through the 2^numNodes potential dependency placements
            for i in range(2**numNodes):
                mark = format(i, 'b').zfill(numNodes) # express placment in binary    
                #print(mark)
                #if dependency is not placed skip.
                if mark == format(0,'b').zfill(numNodes):
                    continue

                marking = list() # A placment equation e.g (1 - a11)*(1 - a12)*a13
                latency = [] # latencies between node hosting primary app and all nodes hosting the dependency for a particular placement. 
                print ("-----NEW MARKING-----")
                # iterate through nodes using the binary placement to generate marking equation
                for i, node in enumerate(nodeList):
                    #print ("mark %s" %mark)
                    #print ("i %s " %i)
                    #print ("mark[i] %s" %type(int(mark[i]))) 
                    # add marking to list and if binary has a 1, add latency to list of possible latencies
                    if int(mark[i]) == 1:
                        marking.append(a2n_df.loc['APP2', node])
                        latency.append(n2n_df.loc[srcNode, node])
                    if int(mark[i]) == 0:
                        marking.append(1 - a2n_df.loc['APP2',node])
                print("latency %s" %latency)
                print("min latency %s" %min(latency))
                print("marking: %s" %marking)
                print("marking * latency: %s" %(z3.Product(marking) * min(latency).item()))
                markings.append(z3.Product(marking) * min(latency).item())

print(markings)
latency_obj = z3.Sum(markings)
pprint.pprint(latency_obj)

    



FSSN3
-----NEW MARKING-----
latency [7]
min latency 7
marking: [1 - a21, 1 - a22, 1 - a23, 1 - a24, a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*(1 - a24)*a25*7
-----NEW MARKING-----
latency [5]
min latency 5
marking: [1 - a21, 1 - a22, 1 - a23, a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*a24*(1 - a25)*5
-----NEW MARKING-----
latency [5, 7]
min latency 5
marking: [1 - a21, 1 - a22, 1 - a23, a24, a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*a24*a25*5
-----NEW MARKING-----
latency [4]
min latency 4
marking: [1 - a21, 1 - a22, a23, 1 - a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*a23*(1 - a24)*(1 - a25)*4
-----NEW MARKING-----
latency [4, 7]
min latency 4
marking: [1 - a21, 1 - a22, a23, 1 - a24, a25]
marking * latency: (1 - a21)*(1 - a22)*a23*(1 - a24)*a25*4
-----NEW MARKING-----
latency [4, 5]
min latency 4
marking: [1 - a21, 1 - a22, a23, a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*a23*a24*(1 - a25)*4
-----NEW MARKING-----
latency [

latency [8, 0, 6]
min latency 0
marking: [a21, a22, a23, 1 - a24, 1 - a25]
marking * latency: a21*a22*a23*(1 - a24)*(1 - a25)*0
-----NEW MARKING-----
latency [8, 0, 6, 9]
min latency 0
marking: [a21, a22, a23, 1 - a24, a25]
marking * latency: a21*a22*a23*(1 - a24)*a25*0
-----NEW MARKING-----
latency [8, 0, 6, 7]
min latency 0
marking: [a21, a22, a23, a24, 1 - a25]
marking * latency: a21*a22*a23*a24*(1 - a25)*0
-----NEW MARKING-----
latency [8, 0, 6, 7, 9]
min latency 0
marking: [a21, a22, a23, a24, a25]
marking * latency: a21*a22*a23*a24*a25*0
FSSN1
-----NEW MARKING-----
latency [5]
min latency 5
marking: [1 - a21, 1 - a22, 1 - a23, 1 - a24, a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*(1 - a24)*a25*5
-----NEW MARKING-----
latency [3]
min latency 3
marking: [1 - a21, 1 - a22, 1 - a23, a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*a24*(1 - a25)*3
-----NEW MARKING-----
latency [3, 5]
min latency 3
marking: [1 - a21, 1 - a22, 1 - a23, a24, a25]
marking * latency:

-----NEW MARKING-----
latency [5, 7, 3]
min latency 3
marking: [a21, a22, a23, 1 - a24, 1 - a25]
marking * latency: a21*a22*a23*(1 - a24)*(1 - a25)*3
-----NEW MARKING-----
latency [5, 7, 3, 6]
min latency 3
marking: [a21, a22, a23, 1 - a24, a25]
marking * latency: a21*a22*a23*(1 - a24)*a25*3
-----NEW MARKING-----
latency [5, 7, 3, 0]
min latency 0
marking: [a21, a22, a23, a24, 1 - a25]
marking * latency: a21*a22*a23*a24*(1 - a25)*0
-----NEW MARKING-----
latency [5, 7, 3, 0, 6]
min latency 0
marking: [a21, a22, a23, a24, a25]
marking * latency: a21*a22*a23*a24*a25*0
FSSN4
-----NEW MARKING-----
latency [0]
min latency 0
marking: [1 - a21, 1 - a22, 1 - a23, 1 - a24, a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*(1 - a24)*a25*0
-----NEW MARKING-----
latency [6]
min latency 6
marking: [1 - a21, 1 - a22, 1 - a23, a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*a24*(1 - a25)*6
-----NEW MARKING-----
latency [6, 0]
min latency 0
marking: [1 - a21, 1 - a22, 1 - a23, a24, a

(1 - a21)*(1 - a22)*(1 - a23)*(1 - a24)*a25*7 +
(1 - a21)*(1 - a22)*(1 - a23)*a24*(1 - a25)*5 +
(1 - a21)*(1 - a22)*(1 - a23)*a24*a25*5 +
(1 - a21)*(1 - a22)*a23*(1 - a24)*(1 - a25)*4 +
(1 - a21)*(1 - a22)*a23*(1 - a24)*a25*4 +
(1 - a21)*(1 - a22)*a23*a24*(1 - a25)*4 +
(1 - a21)*(1 - a22)*a23*a24*a25*4 +
(1 - a21)*a22*(1 - a23)*(1 - a24)*(1 - a25)*8 +
(1 - a21)*a22*(1 - a23)*(1 - a24)*a25*7 +
(1 - a21)*a22*(1 - a23)*a24*(1 - a25)*5 +
(1 - a21)*a22*(1 - a23)*a24*a25*5 +
(1 - a21)*a22*a23*(1 - a24)*(1 - a25)*4 +
(1 - a21)*a22*a23*(1 - a24)*a25*4 +
(1 - a21)*a22*a23*a24*(1 - a25)*4 +
(1 - a21)*a22*a23*a24*a25*4 +
a21*(1 - a22)*(1 - a23)*(1 - a24)*(1 - a25)*0 +
a21*(1 - a22)*(1 - a23)*(1 - a24)*a25*0 +
a21*(1 - a22)*(1 - a23)*a24*(1 - a25)*0 +
a21*(1 - a22)*(1 - a23)*a24*a25*0 +
a21*(1 - a22)*a23*(1 - a24)*(1 - a25)*0 +
a21*(1 - a22)*a23*(1 - a24)*a25*0 +
a21*(1 - a22)*a23*a24*(1 - a25)*0 +
a21*(1 - a22)*a23*a24*a25*0 +
a21*a22*(1 - a23)*(1 - a24)*(1 - a25)*0 +
a21*a22*(1 - a23)*(1 - a24)*

In [13]:
#minimize total resource consumption
costs = list()
for app in appList:
    cost = a2n_df.loc[app]*(rpa_df.loc[:, app]).item()
    #print("cost %s" %cost)
    costs.append(sum(cost))
total_cost = sum(costs)
print(total_cost)

1280 + 0 + a21*128 + a22*128 + a23*128 + a24*128 + a25*128


# The optimizer

## Normalize

In [26]:


min_latency = 0
max_latency = max([sum(n2n_df.loc[node, :]) for node in nodeList])
scale_latency = 1/(max_latency - min_latency)
print("max latency: %s" %max_latency)
print("scale latency: %s " %scale_latency)
print(max_latency * scale_latency)

    
#min_rsrc = in nlappList
max_rsrc = max([sum(rpa_df.loc[rsrc, :]) for rsrc in resourceTypes])*len(nodeList)
print("max_rsrc: %s" %max_rsrc)
min_rsrc = rpa_df.loc['memory', 'APP2' ] + 5*rpa_df.loc['memory', 'APP1']
scale_rsrc = 1/(max_rsrc - min_rsrc)
print("min_rsrc: %s" %min_rsrc)
print("scale rsrc: %s " %scale_rsrc)
print("max_rsrc * scale = %s" %(max_rsrc * scale_rsrc))

print("total_cost %s " %(total_cost * scale_rsrc))



max latency: 30
scale latency: 0.0333333333333 
1.0
max_rsrc: 1920
min_rsrc: 1408
scale rsrc: 0.001953125 
max_rsrc * scale = 3.75
total_cost (1280 + 0 + a21*128 + a22*128 + a23*128 + a24*128 + a25*128)*
0 


In [15]:

opt = z3.Optimize()
constraints = a2n_domain + rsrc_constraint + mdd
#opt.add(constraints)
opt.assert_exprs(constraints)
p = 1

opt.minimize(p*latency_obj*scale_latency + (1-p)*total_cost*scale_rsrc)
#opt.minimize(total_cost)
#opt.maximize(m)
opt.check()

opt_model = opt.model()
print("optimization model \n %s \n" %opt_model)

a2n_df_r = a2n_df.copy()
for app in nlappList:
    #print('App %s' %app)
    for node in nodeList:
        #print('Node %s' %node)
        a2n_df_r.loc[app, node] = opt_model.eval(a2n_df.loc[app, node])
#for app in appList:
#    for node in nodeList:
#        if a2n_df.loc[app,node] in opt_model:
#            print (a2n_df.loc[app,node])
        
print("app placement \n %s \n" %a2n_df_r)  

#Verify the results. 
print("latency cost: %s" %opt_model.eval(latency_obj*p))
print("resource cost: %s" %opt_model.eval(total_cost))

optimization model 
 [a21 = 0, a22 = 0, a23 = 1, a24 = 0, a25 = 0] 

app placement 
      FSSN3 FSSN5 FSSN1 FSSN2 FSSN4
APP1     1     1     1     1     1
APP2     0     0     1     0     0 

latency cost: 18
resource cost: 1408


In [16]:
# Compare alternative to networkx
#https://plot.ly/python/igraph-networkx-comparison/
import networkx as nx
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

G = nx.Graph()

for node1 in nodeList:
    print(dict(rpn_df.loc[:, node]))
    G.add_node(node1, attr_dict=dict(rpn_df.loc[:, node1]))
    id1 = data[node1]['id']
    for node2 in nodeList:
        id2 = data[node2]['id']
        if id1 != id2:            
            G.add_edge(node1, node2)
        
#pprint.pprint(G.edges())
#pprint.pprint(G.nodes(data=True))

#circular_layout(G, dim=2, scale=1, center=None)
pos = nx.circular_layout(G)

#https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.classes.function.set_node_attributes.html
nx.set_node_attributes(G, 'pos', pos)
nx.set_node_attributes(G, 'color', 65)

#nx.get_node_attributes(G)
pprint.pprint(G.nodes(data=True))

{'memory': 512}
{'memory': 512}
{'memory': 512}
{'memory': 512}
{'memory': 512}
[('FSSN3', {'color': 65, 'memory': 512, 'pos': array([ 1.,  0.])}),
 ('FSSN2',
  {'color': 65, 'memory': 512, 'pos': array([ 0.30901699,  0.95105652])}),
 ('FSSN5',
  {'color': 65, 'memory': 512, 'pos': array([-0.80901699,  0.58778525])}),
 ('FSSN1',
  {'color': 65, 'memory': 1024, 'pos': array([-0.80901699, -0.58778525])}),
 ('FSSN4',
  {'color': 65, 'memory': 512, 'pos': array([ 0.30901699, -0.95105652])})]


In [17]:

edge_trace = go.Scatter(
    x=[],
    y=[],
    line=go.Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines+text',
    text=[],
    #textposition='middleright'
)

edge_annotations = go.Annotations()
for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    mx, my = [(x1+x0)*.5,(y1+y0)*.5]
    edge_annotations.append(go.Annotation(
        x = mx,
        y = my, 
        xref='x',
        yref='y',
        text = n2n_df.loc[edge]))
    print("edge: %s lat: %s" %(edge, n2n_df.loc[edge]))
    print("mx: %s, my: %s" %(mx, my))
    print("mx: %s, my: %s" %(mx, my))

edge: ('FSSN3', 'FSSN2') lat: 5
mx: 0.654508497187, my: 0.475528258148
mx: 0.654508497187, my: 0.475528258148
edge: ('FSSN3', 'FSSN5') lat: 8
mx: 0.0954915028125, my: 0.293892626146
mx: 0.0954915028125, my: 0.293892626146
edge: ('FSSN3', 'FSSN1') lat: 4
mx: 0.0954915028125, my: -0.293892626146
mx: 0.0954915028125, my: -0.293892626146
edge: ('FSSN3', 'FSSN4') lat: 7
mx: 0.654508497187, my: -0.475528258148
mx: 0.654508497187, my: -0.475528258148
edge: ('FSSN2', 'FSSN5') lat: 7
mx: -0.25, my: 0.769420884294
mx: -0.25, my: 0.769420884294
edge: ('FSSN2', 'FSSN1') lat: 3
mx: -0.25, my: 0.181635632001
mx: -0.25, my: 0.181635632001
edge: ('FSSN2', 'FSSN4') lat: 6
mx: 0.309016994375, my: -5.55111512313e-17
mx: 0.309016994375, my: -5.55111512313e-17
edge: ('FSSN5', 'FSSN1') lat: 6
mx: -0.809016994375, my: 1.11022302463e-16
mx: -0.809016994375, my: 1.11022302463e-16
edge: ('FSSN5', 'FSSN4') lat: 9
mx: -0.25, my: -0.181635632001
mx: -0.25, my: -0.181635632001
edge: ('FSSN1', 'FSSN4') lat: 5
mx: -0

In [18]:

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers+text',
    hoverinfo='text',
    textposition='bottom',
    marker=go.Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='RdBu',
        reversescale=True,
        color=[],
        size=[]
        #colorbar=dict(
        #    thickness=15,
        #    title='Node Connections',
        #    xanchor='left',
        #    titleside='right'
        #),
        #line=dict(width=2)
    )
)

#print(type(rpa_df))
print(rpa_df.loc[:, 'APP2'])
print(dict(rpa_df.loc[:, 'APP2']))

#for node in G.nodes():
#    type(rpn_df.loc[:,node])
    


memory    128
Name: APP2, dtype: int64
{'memory': 128}


In [19]:
#get positions for apps 
appColor = {}
for node in nodeList:
    numApps = 1
    appG = nx.Graph()
    for app in appList:
        if type(a2n_df_r.loc[app, node]) ==z3.IntNumRef:
            val = (a2n_df_r.loc[app, node]).as_long()
        else:
            val = a2n_df_r.loc[app, node]
        if val == 1:
            if app in appColor:
                color = appColor[app]
            else:
                color = 65+10*numApps
                appColor = {app: color}
            print("app: %s, node:%s, value:%s" %(app, node, val))
            appG.add_node(app+node, attr_dict=dict(rpa_df.loc[:, app]), color=color)
            appG.add_edge(node, app+node) #adding the edge also adds the node
            numApps += 1
            
    #circular_layout(G, dim=2, scale=1, center=None) 
    print("appG nodes: %s" %appG.nodes())
    app_pos = nx.circular_layout(appG, scale=.05, center=G.node[node]['pos'])
    nx.set_node_attributes(appG, 'pos', app_pos)
    
    #pprint.pprint(nx.get_node_attributes(G, 'pos'))
    
    #compose(G, H, name=None)
    # Attributes from H take precedent over attributes from G.
    G = nx.compose(appG, G) 
    #G.add_nodes_from(appG.nodes())
    #pprint.pprint(nx.get_node_attributes(G, 'pos'))


nx.get_node_attributes(G, name='memory')
G.nodes(data=True)

app: APP1, node:FSSN3, value:1
appG nodes: ['FSSN3', 'APP1FSSN3']
app: APP1, node:FSSN5, value:1
appG nodes: ['APP1FSSN5', 'FSSN5']
app: APP1, node:FSSN1, value:1
app: APP2, node:FSSN1, value:1
appG nodes: ['APP1FSSN1', 'FSSN1', 'APP2FSSN1']
app: APP1, node:FSSN2, value:1
appG nodes: ['FSSN2', 'APP1FSSN2']
app: APP1, node:FSSN4, value:1
appG nodes: ['APP1FSSN4', 'FSSN4']


[('APP1FSSN1',
  {'color': 75, 'memory': 256, 'pos': array([-0.75901699, -0.58778525])}),
 ('APP1FSSN3',
  {'color': 75,
   'memory': 256,
   'pos': array([  9.50000000e-01,   6.12323400e-18])}),
 ('FSSN2',
  {'color': 65, 'memory': 512, 'pos': array([ 0.30901699,  0.95105652])}),
 ('APP1FSSN5',
  {'color': 75, 'memory': 256, 'pos': array([-0.75901699,  0.58778525])}),
 ('FSSN4',
  {'color': 65, 'memory': 512, 'pos': array([ 0.30901699, -0.95105652])}),
 ('FSSN3', {'color': 65, 'memory': 512, 'pos': array([ 1.,  0.])}),
 ('FSSN5',
  {'color': 65, 'memory': 512, 'pos': array([-0.80901699,  0.58778525])}),
 ('FSSN1',
  {'color': 65, 'memory': 1024, 'pos': array([-0.80901699, -0.58778525])}),
 ('APP1FSSN4',
  {'color': 75, 'memory': 256, 'pos': array([ 0.35901699, -0.95105652])}),
 ('APP1FSSN2',
  {'color': 75, 'memory': 256, 'pos': array([ 0.25901699,  0.95105652])}),
 ('APP2FSSN1',
  {'color': 85, 'memory': 128, 'pos': array([-0.83401699, -0.63108652])})]

In [20]:
import random

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]
    
#Gt.nodes()

#print(G.adjacency_list())

#for node, adjacencies in enumerate(G.adjacency_list()):
#    print("node: %s, adj: %s" %(node, adjacencies))
#    #node_trace['marker']['color'].append(len(adjacencies))
#    node_info = 'name: '
#    node_trace['text'].append(node_info)

for node in reversed(sorted(G.nodes())):
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    degree = nx.degree(G, node)
    node_info = node + ": " + str(G.node[node]['memory'])
    node_trace['text'].append(node_info)
    node_trace['marker']['size'].append(15*degree)
    node_trace['marker']['color'].append(G.node[node]['color'])

In [21]:
fig = go.Figure(data=go.Data([edge_trace, node_trace]),
             layout=go.Layout(
                 title='<br>App placement graph',
                 titlefont=dict(size=16),
                 showlegend=False,
                 hovermode='closest',
                 margin=dict(b=20,l=5,r=5,t=40),
                 #annotations=[ dict(
                 #    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                 #    showarrow=False,
                 #    xref="paper", yref="paper",
                 #    x=0.005, y=-0.002 ) ],
                 annotations=edge_annotations,
                 xaxis=go.XAxis(showgrid=False, zeroline=False, showticklabels=False),
                 yaxis=go.YAxis(showgrid=False, zeroline=False, showticklabels=False)))

#py.iplot(fig, filename='networkx')
py.plot(fig, filename='networkx.html')

'file:///home/riaps/projects/sandbox/Z3/networkx.html'

In [22]:
#import networx to color graph
# 5 nodes
# primary deployed everywhere
#weight optimization criteria
#visualize
#write up

In [23]:
#minimize latency equation
# App is on FSSN2 and Dependency is deployed once
m21 = a2n_df.loc['APP1','FSSN2'] \
    * (a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1']]).item()

m22 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    * (    a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN2']]).item()
        
m23 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    * (    a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN3']]).item()

# App is on FSSN2 and Dependency is deployed twice
m21_22 = a2n_df.loc['APP1','FSSN2'] \
    *     (a2n_df.loc['APP2','FSSN1']) \
    *     (a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN2']]).item()
    
m21_23 = a2n_df.loc['APP1','FSSN2'] \
    *     (a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    *     (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()
    
m22_23 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    *     (a2n_df.loc['APP2','FSSN2']) \
    *     (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN2'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()

# App is on FSSN2 and Dependency is deployed three times    
m21_22_23 = a2n_df.loc['APP1','FSSN2'] \
    * (a2n_df.loc['APP2','FSSN1']) \
    * (a2n_df.loc['APP2','FSSN2']) \
    * (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN2'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()

m = m21+m22+m23+m21_22+m21_23+m22_23+m21_22_23
m

#a2n_df.loc[app,node] * a2n_df.loc[dep,node]
#a2a_df.loc[app] -> [1, 0]


1*a23*(1 - a24)*(1 - a21)*3 +
1*(1 - a23)*a24*(1 - a21)*0 +
1*(1 - a23)*(1 - a24)*a21*5 +
1*a23*a24*(1 - a21)*0 +
1*a23*(1 - a24)*a21*3 +
1*(1 - a23)*a24*a21*0 +
1*a23*a24*a21*0